In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import io
from PIL import Image
import pyarrow as pa
from vilt.config import ex
from vilt.modules import ViLTransformerSS

from vilt.modules.objectives import cost_matrix_cosine, ipot
from vilt.transforms import pixelbert_transform
from vilt.datamodules.datamodule_base import get_pretrained_tokenizer

In [2]:
df = pa.ipc.RecordBatchFileReader(
    pa.memory_map(f"dataset/normal_cap/cosmos_test.arrow", "r")
).read_all().to_pandas()

In [3]:
from sentence_transformers import SentenceTransformer, util, models
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('stsb-roberta-base-v2')

#Define your train examples. You need more than just two examples...
# train_examples = [
#     InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#     InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

# #Define your train dataset, the dataloader and the train loss
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss = losses.CosineSimilarityLoss(model)

# #Tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

In [4]:
tqdm.pandas()
df['emp_1'] = df['caption_1'].progress_apply(lambda x: model.encode(x[0],convert_to_tensor=True))
df['emp_2'] = df['caption_2'].progress_apply(lambda x: model.encode(x[0],convert_to_tensor=True))

/root/anaconda3/envs/vilt2/lib/python3.7/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1700/1700 [00:26<00:00, 63.76it/s]


In [5]:
df['score'] = df.progress_apply(lambda x: util.cos_sim(x.emp_1, x.emp_2)[0][0], axis=1)
df['label']= df['label'].apply(lambda x:x[0])


100%|██████████| 1700/1700 [00:00<00:00, 5894.75it/s]


In [14]:
max = 0
max_at = 0
confusion_matrix_max = []
for i in np.arange(0.52, 0.55, 0.001):
    df['predict'] = df['score'] < i
    confusion_matrix = pd.crosstab(df['predict'], df['label'], rownames=['Predicted'], colnames=['Actual'])
    result = (confusion_matrix[0][0]+confusion_matrix[1][1])/1700
    if result > max:
        max = result
        max_at = i
        confusion_matrix_max = confusion_matrix
print(max, max_at, confusion_matrix_max)
    

0.7294117647058823 0.526 Actual     False  True 
Predicted              
False        558    168
True         292    682


In [15]:
# 0.5
confusion_matrix = pd.crosstab(df['predict'], df['label'], rownames=['Predicted'], colnames=['Actual'])
print (confusion_matrix)

Actual     False  True 
Predicted              
False        585    214
True         265    636


In [16]:
(585     + 636)/1700

0.7182352941176471

In [12]:
# 0.6
confusion_matrix = pd.crosstab(df['predict'], df['label'], rownames=['Predicted'], colnames=['Actual'])
print (confusion_matrix)

Actual     False  True 
Predicted              
False        407    130
True         443    720


In [24]:
# 0.6
confusion_matrix = pd.crosstab(df['predict'], df['label'], rownames=['Predicted'], colnames=['Actual'])
print (confusion_matrix)

Actual     False  True 
Predicted              
False        407    130
True         443    720


In [22]:
(632 + 416)/1700,(564+535) /1700,(407+720)/1700,(268+802)/1700
#0.4 0.5 0.6 0.7

(0.6164705882352941,
 0.6464705882352941,
 0.6629411764705883,
 0.6294117647058823)

In [33]:
pd.set_option('display.max_colwidth', None)

df.loc[(df['label']==False) & (df['predict']==True),['caption_1','caption_2','score']]

,caption_1,caption_2,score
0,"[PERSON at his announcement in GPE, GPE, on DATE. Mr. PERSON, the former secretary of housing and urban development, would be one of the youngest presidents if elected.]","[PERSON at his announcement in GPE, GPE, on DATE.]","tensor(0.5605, device='cuda:0')"
1,[Supporters of GPE's ruling ORG party come out on DATE to celebrate their candidate's victory in the disputed GPE presidential election],"[A person sits on a truck as supporters of the ruling ORG) celebrate the victory of their candidate in the Zanzibar Presidential election on the outskirts of GPE, on DATE.]","tensor(0.4889, device='cuda:0')"
4,[Men from the LOC tribe perform a traditional jumping ritual as they observe a rite of passage to mark the transition to cultural junior elder within the Masai-Mara national reserve.],"[And on DATE in GPE's Narok county, young PERSON men take part in initiation rites to become moran - the men who are traditionally the warrior class.]","tensor(0.4662, device='cuda:0')"
9,[A man brushes the mouth of a sarcophagus.],"[CARDINAL statues of ancient deities and funerary masks were also discovered at the site, many of which still with their original colours and designs preserved]","tensor(0.1699, device='cuda:0')"
11,"[In GPE, tuk-tuks that used to transport tourists around the city are seen idle because of travel bans and border closures during the pandemic.]",[Tuk-tuk car parked in GPE because there are no tourists],"tensor(0.5748, device='cuda:0')"
...,...,...,...
1687,[PERSON taking a photo],[A child holds a camera next to a destroyed bus],"tensor(0.4245, device='cuda:0')"
1691,[A man exercises on a street in GPE],[A man jumps over a skipping rope in a street],"tensor(0.4664, device='cuda:0')"
1693,"[PERSON, left, and PERSON in ""WORK_OF_ART.""]",[PERSON and PERSON],"tensor(0.5449, device='cuda:0')"
1697,[Homeless people living on streets in GPE],[ORG in GPE],"tensor(0.2635, device='cuda:0')"


In [26]:
embeddings1 = model.encode('PERSON and PERSON', convert_to_tensor=True)
embeddings2 = model.encode('A man exercises on a street in GPE', convert_to_tensor=True)
util.cos_sim(embeddings1, embeddings2)

tensor([[0.1500]], device='cuda:0')